In [1]:
from pyscipopt import Model
from pyscipopt import quicksum
import sys
import math
import numpy as np
from collections import Counter
print(sys.executable)
print(sys.version)
print(sys.version_info)

D:\Programming\Python\Python3.6\python.exe
3.6.4 (v3.6.4:d48eceb, Dec 19 2017, 06:54:40) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [2]:
model = Model("OCT Example")

In [3]:
X = [[0., 1.],
     [1., 0.],
     [0., 0.]]

y = [0, 1, 2]

# max_depth
D = 2
# num of nodes
T = int(math.pow(2, D + 1) - 1)
# num of branch nodes
TB = int(T/2)
# num of features
P = len(X[0])
# num of training points
n = len(X)
# number of labels
y_set = set(y)
K = len(y_set)
# hyper parameter-minimum leaf size
Nmin = 1
# complexity parameter 
alpha = 0

In [4]:
def get_parents(t):
    a = t
    parents = []
    while a != 1:
        a = int(a/2)
        parents.append(a)
    return parents

def get_AL(t):
    a = t
    ALs = []
    while a != 1:
        temp = int(a/2)
        if temp == a/2:
            ALs.append(int(temp))   
        a = int(a/2)
    return ALs

def get_AR(t):
    a = t
    ARs = []
    while a != 1:
        temp = int(a/2)
        if temp != a/2:
            ARs.append(int(temp))   
        a = int(a/2)
    return ARs

In [5]:
Y = np.zeros((n, K))
for i in range(1, n + 1):
    for k in range(1, K + 1):
        Y[i - 1, k - 1] = 1 if y[i - 1] == k - 1 else -1 

In [6]:
# p * 1
eps = [0] * P
X_np = np.array(X)

for j in range(1, P + 1):
    sorted_features = np.sort(X_np[:, j - 1])
    min_diff = float("inf")
    for i in range(2, len(sorted_features)):
        if sorted_features[i] != sorted_features[i-1]:
            min_diff = min(min_diff, sorted_features[i] - sorted_features[i-1])
    eps[j - 1] = min_diff
print(eps)

[1.0, 1.0]


In [7]:
counter = Counter(y)
L_hat = n - counter.most_common(1)[0][1]

In [8]:
a = {}
for j in range(1, P + 1):
    for t in range(1, TB + 1):
        a[j,t] = model.addVar(vtype="B", name="a(%s,%s)" % (j, t))

In [9]:
# b = np.zeros(TB)
b = {}
for t in range(1, TB + 1):
    b[t] = model.addVar(vtype="C", lb=0.0, name="b(%s)" % (t))
print(b)

{1: b(1), 2: b(2), 3: b(3)}


In [10]:
d = {}
for t in range(1, TB + 1):
    d[t] = model.addVar(vtype="B", name="d(%s)" % (t))
print(d)

{1: d(1), 2: d(2), 3: d(3)}


In [11]:
z = {}
for i in range(1, n + 1):
    for t in range(TB + 1, T + 1):
        z[i,t] = model.addVar(vtype="B", name="z(%s,%s)" % (i,t))

In [12]:
l = {}
for t in range(TB + 1, T + 1):
    l[t] = model.addVar(vtype="B", name="l(%s)" % (t))
print(l)

{4: l(4), 5: l(5), 6: l(6), 7: l(7)}


In [13]:
c = {}
for k in range(1, K + 1):
    for t in range(TB + 1, T + 1):
        c[k,t] = model.addVar(vtype="B", name="c(%s,%s)" % (k,t))

In [14]:
Nkt = {}
for k in range(1, K + 1):
    for t in range(TB + 1, T + 1):
        Nkt[k, t] = model.addVar(vtype="I", name="Nkt(%s,%s)" % (k, t))

In [15]:
Nt = {}
for t in range(TB + 1, T + 1):
    Nt[t] = model.addVar(vtype="I", name="Nt(%s)" % (t))

In [16]:
L = {}
for t in range(TB + 1, T + 1):
    L[t] = model.addVar(vtype="I", lb=0, name="L(%s)" % (t))

In [17]:
for t in range(2, TB + 1):
    parent_index = get_parents(t)[0]
#     print(t, d_parent, parent_index)
    model.addCons(d[t] <= d[parent_index], name="DCons(%s)" % t)

In [18]:
for t in range(1, TB + 1):
    model.addCons(b[t] <= d[t], name="BCons(%s)" % t)

In [19]:
for t in range(1, TB + 1):
    model.addCons(quicksum(a[j,t] for j in range(1, P + 1)) == d[t], name="aCons(%s)" % t)

In [20]:
for t in range(TB + 1, T + 1):
    model.addCons(quicksum(z[i,t] for i in range(1, n + 1)) >= Nmin * l[t] , name="zCons(%s)" % t)

In [21]:
for i in range(1, n + 1):
    for t in range(TB + 1, T + 1):
        model.addCons(z[i, t] <= l[t], name="zCons2(%s, %s)" % (i, t))

In [22]:
for i in range(1, n + 1):
    model.addCons(quicksum(z[i,t] for t in range(TB + 1, T + 1)) == 1, name="zCons3(%s)" % i)

In [23]:
for t in range(TB + 1, T + 1):
    model.addCons(quicksum(c[k,t] for k in range(1, K + 1)) == l[t], name="cCons(%s)" % t)

In [24]:
for t in range(TB + 1, T + 1):
    Nt[t] = quicksum(z[i, t] for i in range(1, n + 1))

In [25]:
for k in range(1, K + 1):
    for t in range(TB + 1, T + 1):
        Nkt[k, t] = 0
        for i in range(1, n + 1):
             Nkt[k, t] += (1 + Y[i - 1, k - 1]) * z[i, t]
        Nkt[k, t] /= 2 

In [26]:
for i in range(1, n + 1):
    for t in range(TB + 1, T + 1):
        AL = get_AL(t)
        for m in AL:
            left = 0
            for j in range(1, P + 1):
                left +=  X_np[i-1, j-1] * a[j, m]
            left += min(eps)
            right = b[m] + (1 + max(eps))*(1 - z[i, t])
            model.addCons(left <= right, name="AlCons(%s, %s, %s)" % (i, t, m))        

In [27]:
for i in range(1, n + 1):
    for t in range(TB + 1, T + 1):
        AR = get_AR(t)
        for m in AR:
            left = 0
            for j in range(1, P + 1):
                left += X_np[i-1, j-1] * a[j, m]
            right = b[m] - (1 - z[i, t])
            model.addCons(left >= right, name="ArCons(%s, %s, %s)" % (i, t, m))   

In [28]:
for t in range(TB + 1, T + 1):
    for k in range(1, K + 1):
        model.addCons(L[t] <= Nt[t] - Nkt[k, t] + n * c[k, t], name="LCons1(%s, %s)" % (t, k))        

In [29]:
for t in range(TB + 1, T + 1):
    for k in range(1, K + 1):
        model.addCons(L[t] >= Nt[t] - Nkt[k, t] - n * (1 - c[k, t]), name="LCons2(%s, %s)" % (t, k))        

In [30]:
model.setObjective((1/L_hat) * quicksum(L[t] for t in range(TB + 1, T + 1)) + alpha * quicksum(d[t] for t in range(1, TB + 1)), "minimize")

In [31]:
model.optimize()

In [32]:
if model.getStatus() == "optimal":
    print("Optimal value:", model.getObjVal())
print(model.getStatus())

Optimal value: 0.0
optimal


In [33]:
for i in range(0, D + 1):
    print("Depth ", i + 1)
    for t in range(2**i, 2**(i+1)):
        if i != D: # Branch
            for j in range(1, P+1):
                if model.getVal(a[j, t]) == 1:
                    print("Branch %d, Split Attribute %d, Split Threshold %.3f" %(t, j, model.getVal(b[t])))
                    break
        else:   # Leaf
            valuemax = -float('inf')
            argmax = -10
            for k in range(1, K + 1):
                if model.getVal(Nkt[k, t]) > valuemax:
                    valuemax = model.getVal(Nkt[k, t])
                    argmax = k
            print("Leaf %d, Class Label %d, Point Count %d, Loss %d" %(t, argmax, model.getVal(Nt[t]), model.getVal(L[t])))
    print()

Depth  1
Branch 1, Split Attribute 2, Split Threshold 1.000

Depth  2
Branch 2, Split Attribute 1, Split Threshold 1.000
Branch 3, Split Attribute 2, Split Threshold 0.000

Depth  3
Leaf 4, Class Label 3, Point Count 1, Loss 0
Leaf 5, Class Label 2, Point Count 1, Loss 0
Leaf 6, Class Label 1, Point Count 0, Loss 0
Leaf 7, Class Label 1, Point Count 1, Loss 0

